In [28]:
import numpy as np
import sys
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import interferometer as itf
from scipy.linalg import expm, logm
sys.path.append('../') # Add parent directory to the system path
from boson_sampling_probabilities import output_probability

In [29]:
# This import requires previous directory to be appended to sys.path
from direct_decomposition import direct_decomposition, random_unitary

In [30]:
# Make a random unitary R to simulate
R = random_unitary(3)
print(R)

# R = np.array([[0,1,0],[1,0,0],[0,0,-1]])

[[-0.86173606-0.17633384j -0.43668255+0.16941504j  0.08285484-0.00770073j]
 [ 0.31195192-0.35171644j -0.10689362+0.85745433j  0.17580472-0.03768381j]
 [-0.04976675+0.05312393j  0.17718361-0.05058345j  0.69679621+0.68936452j]]


In [31]:
output_states = [[2,0,0],[0,2,0],[0,0,2],[1,1,0],[1,0,1],[0,1,1]]
for state in output_states:
    print(f"P {state} = {output_probability([0,1,1], state, R)}")

P [2, 0, 0] = 0.0030382549197739616
P [0, 2, 0] = 0.04827473288154631
P [0, 0, 2] = 0.0652400355375547
P [1, 1, 0] = 0.019225928483579232
P [1, 0, 1] = 0.201014895910273
P [0, 1, 1] = 0.6632061522672726


In [32]:
def num_ones(string):
    count = 0
    for i in string:
        if i == "1":
            count += 1
    return count


def run_interferometer_circuit(U, initial_state, num_shots):
    num_photons = num_ones(initial_state)
    num_modes = int(np.shape(U)[0])
    qubits_per_mode = int(np.ceil(np.log2(num_photons + 1)))
    total_num_qubits = qubits_per_mode * num_modes
    circuit = QuantumCircuit(total_num_qubits)
    
    circuit.initialize(initial_state)
    interferom = direct_decomposition(U, num_photons)
    circuit.compose(interferom, qubits=list(range(total_num_qubits)), inplace=True)
    circuit.measure_all()

    simulator = AerSimulator()
    circuit = transpile(circuit, simulator)
    result = simulator.run(circuit, shots=num_shots).result()
    counts = result.get_counts(circuit)

    probs = dict()
    for key in counts.keys():
        probs[key] = counts[key] / num_shots
    return probs

In [33]:
probs = run_interferometer_circuit(R, "000101", 1e5)

In [34]:
# print(probs)
ordered_measurements = ["000010","001000","100000","000101","010001","010100"]
for key in ordered_measurements:
    print(probs[key])

0.27392
0.26906
0.00042
0.41828
0.02528
0.01304


In [35]:
output_states = [[2,0,0],[0,2,0],[0,0,2],[1,1,0],[1,0,1],[0,1,1]]
for state in output_states:
    print(f"P {state} = {output_probability([1,1,0], state, R)}")

P [2, 0, 0] = 0.3394812896894531
P [0, 2, 0] = 0.33004870238373185
P [0, 0, 2] = 0.0003598227872447441
P [1, 1, 0] = 0.29157823387426013
P [1, 0, 1] = 0.02253496076737068
P [0, 1, 1] = 0.015996990497939182
